# 寻找轮廓

### 导入资源并显示图像

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

# Read in the image
image = cv2.imread('images/thumbs_up_down.jpg')

# Change color to RGB (from BGR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)

In [ ]:
<matplotlib.image.AxesImage at 0x117e5a6d8>




![png](output_2_1.png)


### 生成一个用于查找轮廓的二值图像

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

# Create a binary thresholded image
retval, binary = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)

plt.imshow(binary, cmap='gray')


In [ ]:
<matplotlib.image.AxesImage at 0x11a0ed898>




![png](output_4_1.png)


### 查找并绘出轮廓

In [ ]:
# Find contours from thresholded, binary image
retval, contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw all contours on a copy of the original image
contours_image = np.copy(image)
contours_image = cv2.drawContours(contours_image, contours, -1, (0,255,0), 3)

plt.imshow(contours_image)

In [ ]:
<matplotlib.image.AxesImage at 0x11a2b2a58>




![png](output_6_1.png)


## 轮廓特征

每个轮廓都有许多可以计算的特征，包括轮廓的面积、方向（即大部分轮廓指向的方向）、周长以及 [OpenCV文档中](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html)概述的许多其他属性。

在下一个单元格中，你需要识别左手和右手轮廓的方向。这些方向可以让你知道哪只手的拇指向上，哪只手的拇指朝下！

### 方向

一个对象的方向是指该对象所指向的角度。要找到轮廓的角度，首先应找到与该轮廓拟合的椭圆，然后从该椭圆中提取`angle`。

In [ ]:
# Fit an ellipse to a contour and extract the angle from that ellipse
(x,y), (MA,ma), angle = cv2.fitEllipse(selected_contour)

**方向值**

这些方向值以度为单位，从x轴测量。值为零表示平直线，值为90表示该轮廓指向竖直方向！

因此，你为每个轮廓计算的方向角度都应该能够告诉我们关于手掌大概位置的信息。拇指向上的手应该比拇指向下的手角度更高（接近90度）。

### TODO: 找到每个轮廓的方向

In [ ]:
## TODO: Complete this function so that 
## it returns the orientations of a list of contours
## The list should be in the same order as the contours
## i.e. the first angle should be the orientation of the first contour
def orientations(contours):
    """
    Orientation 
    :param contours: a list of contours
    :return: angles, the orientations of the contours
    """
    
    # Create an empty list to store the angles in
    # Tip: Use angles.append(value) to add values to this list
    angles = []
    
    for cnt in contours:
        # Fit an ellipse to a contour and extract the angle from that ellipse
        (x,y), (MA,ma), angle = cv2.fitEllipse(cnt)
        angles.append(angle)
        
    return angles


# ---------------------------------------------------------- #
# Print out the orientation values
angles = orientations(contours)
print('Angles of each contour (in degrees): ' + str(angles))

In [ ]:
Angles of each contour (in degrees): [61.274147033691406, 84.45772552490234]


### 边界矩形

在下一个单元格中，你需要找到*左* 手轮廓周围的边界矩形，其中，该手的拇指向上。然后，使用该边界矩形裁剪图像，之后找到那一只手轮廓周围的边界矩形！

In [ ]:
# Find the bounding rectangle of a selected contour
x,y,w,h = cv2.boundingRect(selected_contour)

# Draw the bounding rectangle as a purple box
box_image = cv2.rectangle(contours_image, (x,y), (x+w,y+h), (200,0,200),2)

要裁剪图像，请选择要包含的图像的正确宽度和高度。

In [ ]:
# Crop using the dimensions of the bounding rectangle (x, y, w, h)
cropped_image = image[y: y + h, x: x + w] 

### TODO: 围绕轮廓裁剪图像

In [ ]:
## TODO: Complete this function so that
## it returns a new, cropped version of the original image
def left_hand_crop(image, selected_contour):
    """
    Left hand crop 
    :param image: the original image
    :param selectec_contour: the contour that will be used for cropping
    :return: cropped_image, the cropped image around the left hand
    """
    
    ## TODO: Detect the bounding rectangle of the left hand contour
    
    ## TODO: Crop the image using the dimensions of the bounding rectangle
    # Make a copy of the image to crop
    cropped_image = np.copy(image)
    
    # Find the bounding rectangle of a selected contour
    x,y,w,h = cv2.boundingRect(selected_contour)
    
    # Crop the image using the dimensions of the bounding rectangle
    cropped_image = cropped_image[y: y + h, x: x + w]
    
    return cropped_image


## TODO: Select the left hand contour from the list
## Replace this value
selected_contour = contours[1]


# ---------------------------------------------------------- #
# If you've selected a contour
if(selected_contour is not None):
    # Call the crop function with that contour passed in as a parameter
    cropped_image = left_hand_crop(image, selected_contour)
    plt.imshow(cropped_image)

![png](output_11_0.png)